UBCF Notebook - Not Confidence-Aware Approach

In [1]:
import sys
sys.path.insert(0,'C:\\Users\\Jacob\\Documents\\GitHub\\lenskit_confidence')

In [2]:
from lenskit.algorithms import user_knn, Recommender
from lenskit.metrics.predict import rmse
from lenskit import datasets
from lenskit.datasets import MovieLens
from lenskit import crossfold as xf
from lenskit import batch, topn, datasets
from lenskit import topn, util
from lenskit.crossfold import partition_users, SampleN, partition_rows
import pandas as pd
import logging
_logger = logging.getLogger()

In [3]:
# Load MovieLens data in expected format
ratings_df = MovieLens('../data/ml-latest-small')
#ratings_df = MovieLens('../data/ml-1m')

In [4]:
util.log_to_notebook()

[   INFO] lenskit.util.log notebook logging configured


In [62]:
all_recs = []
all_preds = []
all_test = []

In [63]:
num_cv = 5
samp = xf.SampleN(5)

In [64]:
pairs = list(partition_rows(ratings_df.ratings, num_cv))
#pairs = list(partition_users(ratings_df.ratings, num_cv, samp))

[   INFO] lenskit.crossfold partitioning 100004 ratings into 5 partitions


In [65]:
#agg_type = 'average'
agg_type = 'weighted-average'
#agg_type = 'sum'

In [66]:
nbhd_sizes = [25] #[10, 25, 50, 100, 150, 250]

In [67]:
for f in nbhd_sizes:
    _logger.info('Starting neighborhood size: %d', f)
    cv_fold = 0
    for train, test in pairs: #: # # 
        
        cv_fold += 1
        _logger.info('cv fold: %d', cv_fold)
        print(cv_fold)
        #train['RunId'] = cv_fold
        test['RunId'] = cv_fold
        
        _logger.info('Training set size: %d | Test set size: %d', len(train), len(test))
        all_test.append(test)
        _logger.info('Configuring algorithm...')
        # configure the algorithm
        ####algo = user_knn.UserUser(nnbrs = f, aggregate = agg_type) # weighted-average # average
        algo = user_knn.UserUser(nnbrs = f, aggregate = agg_type) # Confidence-Aware UBCF
        ####algo = als.BiasedMF(50)
        # make sure it can recommend, not just predict
        algo = Recommender.adapt(algo)
        
        # train the algorithm
        algo.fit(train) # [:-1000]
        
        ###algo.fit(train)
        print('Starting prediction process...')
        # predict ratings
        preds = batch.predict(algo, test)
        preds['nnbhrs'] = f
        preds['RunId'] = cv_fold
        all_preds.append(preds)
        print('Starting recommendation process...')
        # produce recommendations
        ####users = test['user'].unique()
        ####recs = batch.recommend(algo, users, 20)
        ####all_recs.append(recs)
        _logger.info('Finishing neighborhood size: %d', f)

[   INFO] root Starting neighborhood size: 25
[   INFO] root cv fold: 1
1
[   INFO] root Training set size: 80003 | Test set size: 20001
[   INFO] root Configuring algorithm...
[   INFO] lenskit.algorithms.basic trained unrated candidate selector for 80003 ratings
Starting prediction process...
[   INFO] lenskit.sharing.shm serialized TopN/UserUser(nnbrs=25, min_sim=0) to 1121 pickle bytes with 13 buffers of 2429908 bytes
[   INFO] lenskit.util.parallel setting up ProcessPoolExecutor w/ 2 workers
[   INFO] lenskit.batch._predict generating 20001 predictions for 670 users (setup took  18ms)


<ipython-input-67-33223d960b13>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['RunId'] = cv_fold


[   INFO] lenskit.batch._predict generated 20001 predictions for 670 users in 16.57s
Starting recommendation process...
[   INFO] root Finishing neighborhood size: 25
[   INFO] root cv fold: 2
2
[   INFO] root Training set size: 80003 | Test set size: 20001
[   INFO] root Configuring algorithm...
[   INFO] lenskit.algorithms.basic trained unrated candidate selector for 80003 ratings
Starting prediction process...
[   INFO] lenskit.sharing.shm serialized TopN/UserUser(nnbrs=25, min_sim=0) to 1121 pickle bytes with 13 buffers of 2429668 bytes
[   INFO] lenskit.util.parallel setting up ProcessPoolExecutor w/ 2 workers
[   INFO] lenskit.batch._predict generating 20001 predictions for 671 users (setup took  15ms)
[   INFO] lenskit.batch._predict generated 20001 predictions for 671 users in 16.50s
Starting recommendation process...
[   INFO] root Finishing neighborhood size: 25
[   INFO] root cv fold: 3
3
[   INFO] root Training set size: 80003 | Test set size: 20001
[   INFO] root Configuri

In [68]:
##recs = pd.concat(all_recs, ignore_index=True)
preds = pd.concat(all_preds, ignore_index=True)
test = pd.concat(all_test, ignore_index=True)


In [70]:
preds
#train
#test

,user,item,rating,timestamp,RunId,prediction,nnbhrs
0,514,229,3.0,853894449,1,4.019385,25
1,363,322,2.0,938890637,1,3.913314,25
2,457,39446,1.5,1471386365,1,2.553097,25
3,547,1639,0.5,1053173814,1,3.343169,25
4,15,110553,0.5,1416120149,1,2.388490,25
...,...,...,...,...,...,...,...
99999,528,908,4.5,1143757746,5,3.926231,25
100000,254,231,2.0,845157247,5,3.050746,25
100001,160,2375,3.0,974257547,5,3.159098,25
100002,298,101864,4.0,1447368645,5,3.327234,25


In [ ]:
preds.loc[preds['prediction'] > 5,'prediction'] = 5
preds.loc[preds['prediction'] < 1,'prediction'] = 1

#dfd.loc[mask, 'c'] = 42

In [ ]:
preds['prediction'].describe()

In [ ]:
preds[['nnbhrs','var']].groupby(['nnbhrs']).describe()

In [ ]:
preds[['nnbhrs','prediction']].groupby(['nnbhrs']).describe()

In [ ]:
# compute RMSE
print('RMSE:', rmse(preds['prediction'], preds['rating'])) 

# ML100k - partition users - average - RMSE = 0.9157690305188498, 0.91377232782083
# ML100k - partition users - weighted-average - RMSE = 0.9185319353148285, 0.9333875278175761
# ML100k - partition users - sum - RMSE = 1.3963497485089527


In [ ]:
#preds.groupby(['nnbhrs'])['prediction', 'rating'].rmse()
preds['err'] = (preds['prediction'] - preds['rating'])**2

In [ ]:
import numpy as np
np.sqrt(preds[['err', 'nnbhrs']].groupby(['nnbhrs']).mean() )#.groupby(['nnbhrs']).sqrt() #sqrt()
#preds.groupby(['nnbhrs']).sqrt()